# Compare different Berkeley Earth realisations

In [ ]:
import pathlib

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import xarray as xr

from utils import berkeley, utils

## Difference in global mean

In [ ]:
berkeley_globmean_1850_1900_v2024 = berkeley.read_globmean(ref_period=slice(1850, 1900), version="v2024")
berkeley_globmean_1850_1900_v2025 = berkeley.read_globmean(ref_period=slice(1850, 1900), version="v2025")

In [ ]:
berkeley_globmean_1850_1900_v2024.plot()
berkeley_globmean_1850_1900_v2025.plot()

In [ ]:
berkeley_globmean_abs_v2024 = berkeley.read_globmean(ref_period=None, version="v2024")
berkeley_globmean_abs_v2025 = berkeley.read_globmean(ref_period=None, version="v2025")

In [ ]:
berkeley_globmean_abs_v2024.plot()
berkeley_globmean_abs_v2025.plot()

In [ ]:
diff_anom = berkeley_globmean_1850_1900_v2024 - berkeley_globmean_1850_1900_v2025

diff_abs = berkeley_globmean_abs_v2024 - berkeley_globmean_abs_v2025

f, ax = plt.subplots()

diff_anom.plot(ax=ax, label="v2024 - v2025: anom. temp.")
diff_abs.plot(ax=ax, label="v2024 - v2025: abs. temp.")

plt.legend()

ax.axhline(0, lw=0.5, c="0.1")

## Difference in spatial data

In [ ]:
period = slice("1955", "1960")

tmax_v2025 = berkeley.read_full("TMAX", version="v2025")
tmin_v2025 = berkeley.read_full("TMIN", version="v2025")

In [ ]:
tmax_v2024 = berkeley.read_full("TMAX", version="v2024")
tmin_v2024 = berkeley.read_full("TMIN", version="v2024")

In [ ]:
tmax_v2024.climatology.load()
tmax_v2025.climatology.load()
None

In [ ]:
tmin_v2024.climatology.load()
tmin_v2025.climatology.load()
None

In [ ]:
tmax_v2024

In [ ]:
tmax_v2025

### Difference in climatology

In [ ]:
f, ax = plt.subplots()

h = (tmax_v2025.climatology - tmax_v2024.climatology).mean("day_number").plot(ax=ax, add_colorbar=False)

cbar = plt.colorbar(h, ax=ax)

cbar.set_label("climatology difference (°C)")

ax.set_title("Annual mean difference in TMAX climatology (v2025 - v2024)", loc="left", fontsize=8)

# plt.savefig("BerkeleyEarth_TMAX_climatology_diff.png")

In [ ]:
(tmin_v2024.climatology - tmin_v2025.climatology).mean("day_number").plot()

In [ ]:
lon = 8
lat= 47

# (tmax_v2024.climatology - tmax_v2025.climatology).sel(lat=lat, lon=lon, method="nearest").plot()

f, axs = plt.subplots(1, 2, sharey=True)


ax = axs[0]
tmin_v2024.climatology.sel(lat=lat, lon=lon, method="nearest").plot(ax=ax, label="v2024", color="r")
tmin_v2025.climatology.sel(lat=lat, lon=lon, method="nearest").plot(ax=ax, label="v2025", color="b")
ax.set_title("TMIN")
ax.set_ylabel("Temperature (°C)")
ax.legend()

ax = axs[1]
tmax_v2024.climatology.sel(lat=lat, lon=lon, method="nearest").plot(ax=ax, label="v2024", color="r")
tmax_v2025.climatology.sel(lat=lat, lon=lon, method="nearest").plot(ax=ax, label="v2025", color="b")
ax.set_title("TMAX")
ax.set_ylabel("")
ax.legend()

In [ ]:
a = tmin_v2024.climatology.sel(lat=lat, lon=lon, method="nearest")
b = tmax_v2024.climatology.sel(lat=lat, lon=lon, method="nearest")

plt.fill_between(a.day_number, a, b, color="#3182bd88", label="v2024")


a = tmin_v2025.climatology.sel(lat=lat, lon=lon, method="nearest")
b = tmax_v2025.climatology.sel(lat=lat, lon=lon, method="nearest")

plt.fill_between(a.day_number, a, b, color="#c51b8a88", label="v2025")

plt.legend()

plt.title("TX and TN difference")

### Compare TXx

In [ ]:
tmax_seas_v2024 = berkeley.add_climatology(tmax_v2024)

In [ ]:
tmax_seas_v2025 = berkeley.add_climatology(tmax_v2025)

In [ ]:
with xr.set_options(use_flox=True):
    txx_v2024 = tmax_seas_v2024.groupby("time.year").max("time", engine="flox").compute()

In [ ]:
with xr.set_options(use_flox=True):
    txx_v2025 = tmax_seas_v2025.groupby("time.year").max("time", engine="flox").compute()

In [ ]:
(txx_v2025 - txx_v2024).sel(year=2000).plot(robust=True)